
https://www.kaggle.com/gabrio/board-games-dataset/data

In [30]:
import pandas as pd
import sqlite3
conn = sqlite3.connect('database.sqlite')
conn.row_factory = sqlite3.Row
c = conn.cursor()

### Find games of mechanic

In [31]:
FILTER_MECHANIC = '%Worker Placement%'

c.execute('SELECT * FROM BoardGames WHERE "attributes.boardgamemechanic" LIKE ?', [FILTER_MECHANIC])
rows = c.fetchall()
print('Found {} games'.format(len(rows)))

Found 1393 games


In [32]:
c.execute('SELECT * FROM BoardGames WHERE "attributes.boardgamemechanic" LIKE ? AND "stats.subtype.boardgame.pos" IS NOT NULL', [FILTER_MECHANIC])
rows = c.fetchall()
print('Found {} ranked games'.format(len(rows)))

#r[stats.subtype.boardgame.pos] for r in rows

Found 384 ranked games


### Top Games

In [33]:
TOP_N_LIMIT = 10

c.execute('SELECT * FROM BoardGames WHERE "attributes.boardgamemechanic" LIKE ? AND "stats.subtype.boardgame.pos" IS NOT NULL ORDER BY "stats.subtype.boardgame.pos" ASC LIMIT ?', [FILTER_MECHANIC, TOP_N_LIMIT])
rows = c.fetchall()

def create_games(rows):
    games = []
    for r in rows:
        game = {}
        for key in r.keys():
            game[key] = r[key]
        games.append(game)
    return games

games = pd.DataFrame(create_games(rows))
top_games = games[['stats.subtype.boardgame.pos', 'details.name', 'details.yearpublished', 'attributes.boardgamedesigner', 'stats.averageweight', 'details.minplayers', 'details.maxplayers', 'details.minplaytime', 'details.maxplaytime']]
top_games

,stats.subtype.boardgame.pos,details.name,details.yearpublished,attributes.boardgamedesigner,stats.averageweight,details.minplayers,details.maxplayers,details.minplaytime,details.maxplaytime
0,10.0,Caverna: The Cave Farmers,2013,Uwe Rosenberg,3.7886,1,7,30,210
1,13.0,Agricola,2007,Uwe Rosenberg,3.6313,1,5,30,150
2,22.0,Robinson Crusoe: Adventures on the Cursed Island,2012,Ignacy Trzewiczek,3.6942,1,4,90,180
3,27.0,Le Havre,2008,Uwe Rosenberg,3.7603,1,5,30,150
4,32.0,Tzolk'in: The Mayan Calendar,2012,"Simone Luciani,Daniele Tascini",3.6130,2,4,90,90
5,34.0,Viticulture Essential Edition,2015,"Morten Monrad Pedersen,Jamey Stegmaier,Alan Stone",2.9682,1,6,90,90
6,36.0,Keyflower,2012,"Sebastian Bleasdale,Richard Breese",3.3378,2,6,90,120
7,38.0,Caylus,2005,William Attia,3.8210,2,5,60,150
8,39.0,The Voyages of Marco Polo,2015,"Simone Luciani,Daniele Tascini",3.1815,2,4,40,100
9,44.0,Dominant Species,2010,Chad Jensen,4.0275,2,6,120,240
